In [1]:
import os
from torch_geometric.data import Data
import torch

In [2]:
dataset = []

for ptfile in os.listdir('concept_drift_21'):
    dataset.append(torch.load(f'concept_drift_21/{ptfile}'))

In [3]:
sum=0
for i in dataset:
    sum += len(i.y)

In [4]:
sum

3071

In [5]:
train_data = dataset[0:int(0.8*366)]
test_data = dataset[int(0.8*366):]

In [6]:
dataset[1].edge_to_edge_adj_matr.count_nonzero()

tensor(12)

In [7]:
dataset[1].y

tensor([0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0])

In [8]:
dataset[1].edge_attr

tensor([[0.8070, 0.0113, 0.4948, 0.0058, 0.4762, 0.1935, 0.0141, 0.2088, 0.0127,
         0.0098, 0.0071, 0.1852],
        [0.6767, 0.6767, 0.6767, 0.6767, 0.6767, 0.6767, 0.6767, 0.6767, 0.6767,
         0.6767, 0.6767, 0.6767],
        [0.0000, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000,
         1.0000, 1.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000],
        [1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 1.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000,
         0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000]], dtype=torch.float64)

In [9]:
len(train_data)

292

In [10]:
len(test_data)

74

In [11]:
from model_nenn_simplificado_2 import Nenn

In [12]:
dataset[0].x.shape

torch.Size([6, 26])

In [13]:
dataset[0].edge_attr.shape

torch.Size([8, 13])

In [14]:
model = Nenn(6,8,6,8,2)

In [15]:
model = model.to('cpu')

In [16]:
loss = torch.nn.CrossEntropyLoss()
# TODO Undersampling Smote
# TODO Oversampling
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [17]:
dataset[0]

Data(x=[6, 26], edge_index=[2, 13], edge_attr=[8, 13], y=[13], node_to_node_adj_matr=[26, 26], node_to_edge_adj_matr=[26, 13], edge_to_node_adj_matr=[13, 26], edge_to_edge_adj_matr=[13, 13])

In [18]:
def train():
    model.train()
    # Enumerar sobre os dados.
    for epoch in range(100):
        for ts, data in enumerate(train_data):
        
            # Resetar Gradientes
            optimizer.zero_grad()
            # Passando as informações do batch e de conexão dos grafos
            logits, _ = model(
                data.x.T.type(torch.FloatTensor),
                data.edge_attr.T.type(torch.FloatTensor),
                data.edge_to_edge_adj_matr.type(torch.FloatTensor),
                data.edge_to_node_adj_matr.type(torch.FloatTensor),
                data.node_to_edge_adj_matr.type(torch.FloatTensor),
                data.node_to_node_adj_matr.type(torch.FloatTensor)
            )
            #label_pred = pred.max(1)[1]
            # Calculando a perda e os gradientes.
            l = loss(logits, data.y)
            l.backward()
            # Atualizar usando os gradientes.
            optimizer.step()
            print('ts',ts+1,'epoch =', epoch + 1, 'loss =', l.item())

In [19]:
train()

ts 1 epoch = 1 loss = 0.6931944489479065
ts 2 epoch = 1 loss = 0.6931473612785339
ts 3 epoch = 1 loss = 0.6931089162826538
ts 4 epoch = 1 loss = 0.6931473016738892
ts 5 epoch = 1 loss = 0.6934147477149963
ts 6 epoch = 1 loss = 0.6932733654975891
ts 7 epoch = 1 loss = 0.6928371787071228
ts 8 epoch = 1 loss = 0.6931487917900085
ts 9 epoch = 1 loss = 0.6924542784690857
ts 10 epoch = 1 loss = 0.6927671432495117
ts 11 epoch = 1 loss = 0.6925217509269714
ts 12 epoch = 1 loss = 0.692986249923706
ts 13 epoch = 1 loss = 0.6940470933914185
ts 14 epoch = 1 loss = 0.6928256750106812
ts 15 epoch = 1 loss = 0.690710723400116
ts 16 epoch = 1 loss = 0.6904159188270569
ts 17 epoch = 1 loss = 0.6931567788124084
ts 18 epoch = 1 loss = 0.6944693326950073
ts 19 epoch = 1 loss = 0.6940638422966003
ts 20 epoch = 1 loss = 0.693157434463501
ts 21 epoch = 1 loss = 0.689294695854187
ts 22 epoch = 1 loss = 0.6931564807891846
ts 23 epoch = 1 loss = 0.6898234486579895
ts 24 epoch = 1 loss = 0.6938347220420837
ts 25

ts 198 epoch = 1 loss = 0.6683065295219421
ts 199 epoch = 1 loss = 0.6951216459274292
ts 200 epoch = 1 loss = 0.631951093673706
ts 201 epoch = 1 loss = 0.645725429058075
ts 202 epoch = 1 loss = 0.6738725304603577
ts 203 epoch = 1 loss = 0.6844931244850159
ts 204 epoch = 1 loss = 0.646733283996582
ts 205 epoch = 1 loss = 0.6591290235519409
ts 206 epoch = 1 loss = 0.7131246328353882
ts 207 epoch = 1 loss = 0.6624648571014404
ts 208 epoch = 1 loss = 0.6733157634735107
ts 209 epoch = 1 loss = 0.6669183373451233
ts 210 epoch = 1 loss = 0.628728985786438
ts 211 epoch = 1 loss = 0.6507291197776794
ts 212 epoch = 1 loss = 0.6647982001304626
ts 213 epoch = 1 loss = 0.6277869939804077
ts 214 epoch = 1 loss = 0.6857360601425171
ts 215 epoch = 1 loss = 0.6726647019386292
ts 216 epoch = 1 loss = 0.672570526599884
ts 217 epoch = 1 loss = 0.654037356376648
ts 218 epoch = 1 loss = 0.6878370046615601
ts 219 epoch = 1 loss = 0.6536837220191956
ts 220 epoch = 1 loss = 0.6254404783248901
ts 221 epoch = 1 

ts 104 epoch = 2 loss = 0.5884064435958862
ts 105 epoch = 2 loss = 0.5881725549697876
ts 106 epoch = 2 loss = 0.7550451755523682
ts 107 epoch = 2 loss = 0.5876682996749878
ts 108 epoch = 2 loss = 0.6994091868400574
ts 109 epoch = 2 loss = 0.6713730096817017
ts 110 epoch = 2 loss = 0.6512576937675476
ts 111 epoch = 2 loss = 0.7370231747627258
ts 112 epoch = 2 loss = 0.5866159796714783
ts 113 epoch = 2 loss = 0.7372320294380188
ts 114 epoch = 2 loss = 0.7098513245582581
ts 115 epoch = 2 loss = 0.6428451538085938
ts 116 epoch = 2 loss = 0.7099117040634155
ts 117 epoch = 2 loss = 0.667108416557312
ts 118 epoch = 2 loss = 0.6833510398864746
ts 119 epoch = 2 loss = 0.5856741070747375
ts 120 epoch = 2 loss = 0.6425723433494568
ts 121 epoch = 2 loss = 0.6139196753501892
ts 122 epoch = 2 loss = 0.7426543831825256
ts 123 epoch = 2 loss = 0.6104851961135864
ts 124 epoch = 2 loss = 0.6997364163398743
ts 125 epoch = 2 loss = 0.6997560262680054
ts 126 epoch = 2 loss = 0.6421520113945007
ts 127 epoch

ts 13 epoch = 3 loss = 0.7709291577339172
ts 14 epoch = 3 loss = 0.6829598546028137
ts 15 epoch = 3 loss = 0.550965428352356
ts 16 epoch = 3 loss = 0.5509045124053955
ts 17 epoch = 3 loss = 0.7049888968467712
ts 18 epoch = 3 loss = 0.7643615007400513
ts 19 epoch = 3 loss = 0.7436127662658691
ts 20 epoch = 3 loss = 0.7050237059593201
ts 21 epoch = 3 loss = 0.5506120324134827
ts 22 epoch = 3 loss = 0.7050324082374573
ts 23 epoch = 3 loss = 0.5891338586807251
ts 24 epoch = 3 loss = 0.7271467447280884
ts 25 epoch = 3 loss = 0.5500760078430176
ts 26 epoch = 3 loss = 0.7714377641677856
ts 27 epoch = 3 loss = 0.6532692313194275
ts 28 epoch = 3 loss = 0.5888824462890625
ts 29 epoch = 3 loss = 0.5943568348884583
ts 30 epoch = 3 loss = 0.6274846196174622
ts 31 epoch = 3 loss = 0.662792980670929
ts 32 epoch = 3 loss = 0.6118770241737366
ts 33 epoch = 3 loss = 0.6187158226966858
ts 34 epoch = 3 loss = 0.6532836556434631
ts 35 epoch = 3 loss = 0.5491993427276611
ts 36 epoch = 3 loss = 0.62715750932

ts 214 epoch = 3 loss = 0.6839541792869568
ts 215 epoch = 3 loss = 0.647683322429657
ts 216 epoch = 3 loss = 0.6476678848266602
ts 217 epoch = 3 loss = 0.5967205762863159
ts 218 epoch = 3 loss = 0.6900584697723389
ts 219 epoch = 3 loss = 0.5965788960456848
ts 220 epoch = 3 loss = 0.5198469758033752
ts 221 epoch = 3 loss = 0.6591289639472961
ts 222 epoch = 3 loss = 0.7388681769371033
ts 223 epoch = 3 loss = 0.6901415586471558
ts 224 epoch = 3 loss = 0.5192400217056274
ts 225 epoch = 3 loss = 0.7115533351898193
ts 226 epoch = 3 loss = 0.6473738551139832
ts 227 epoch = 3 loss = 0.7116033434867859
ts 228 epoch = 3 loss = 0.5187288522720337
ts 229 epoch = 3 loss = 0.5668603777885437
ts 230 epoch = 3 loss = 0.6472854018211365
ts 231 epoch = 3 loss = 0.5039455890655518
ts 232 epoch = 3 loss = 0.8086426854133606
ts 233 epoch = 3 loss = 0.7118206024169922
ts 234 epoch = 3 loss = 0.6148541569709778
ts 235 epoch = 3 loss = 0.6471646428108215
ts 236 epoch = 3 loss = 0.7630131244659424
ts 237 epoch

...

ts 11 epoch = 98 loss = 0.5091034173965454
ts 12 epoch = 98 loss = 0.31792905926704407
ts 13 epoch = 98 loss = 0.6883962750434875
ts 14 epoch = 98 loss = 0.3137405216693878
ts 15 epoch = 98 loss = 0.31326165795326233
ts 16 epoch = 98 loss = 0.3132616877555847
ts 17 epoch = 98 loss = 0.313725084066391
ts 18 epoch = 98 loss = 0.41984862089157104
ts 19 epoch = 98 loss = 0.3145619332790375
ts 20 epoch = 98 loss = 0.31366947293281555
ts 21 epoch = 98 loss = 0.31326165795326233
ts 22 epoch = 98 loss = 0.32052069902420044
ts 23 epoch = 98 loss = 0.313892662525177
ts 24 epoch = 98 loss = 0.3200913071632385
ts 25 epoch = 98 loss = 0.31326165795326233
ts 26 epoch = 98 loss = 0.32222288846969604
ts 27 epoch = 98 loss = 0.31326165795326233
ts 28 epoch = 98 loss = 0.3136402368545532
ts 29 epoch = 98 loss = 0.3133293688297272
ts 30 epoch = 98 loss = 0.31326165795326233
ts 31 epoch = 98 loss = 0.31803637742996216
ts 32 epoch = 98 loss = 0.3132639229297638
ts 33 epoch = 98 loss = 0.31624239683151245
t

ts 202 epoch = 98 loss = 0.31482645869255066
ts 203 epoch = 98 loss = 0.3992689549922943
ts 204 epoch = 98 loss = 0.31509268283843994
ts 205 epoch = 98 loss = 0.4944012463092804
ts 206 epoch = 98 loss = 0.32115697860717773
ts 207 epoch = 98 loss = 0.3132617175579071
ts 208 epoch = 98 loss = 0.31563299894332886
ts 209 epoch = 98 loss = 0.31347134709358215
ts 210 epoch = 98 loss = 0.4369083046913147
ts 211 epoch = 98 loss = 0.5075146555900574
ts 212 epoch = 98 loss = 0.4755517840385437
ts 213 epoch = 98 loss = 0.31326165795326233
ts 214 epoch = 98 loss = 0.3139798045158386
ts 215 epoch = 98 loss = 0.313290536403656
ts 216 epoch = 98 loss = 0.3138914406299591
ts 217 epoch = 98 loss = 0.31326550245285034
ts 218 epoch = 98 loss = 0.3154389560222626
ts 219 epoch = 98 loss = 0.5582324862480164
ts 220 epoch = 98 loss = 0.31326165795326233
ts 221 epoch = 98 loss = 0.316219300031662
ts 222 epoch = 98 loss = 0.42788559198379517
ts 223 epoch = 98 loss = 0.3204442858695984
ts 224 epoch = 98 loss = 

ts 100 epoch = 99 loss = 0.31326165795326233
ts 101 epoch = 99 loss = 0.31602412462234497
ts 102 epoch = 99 loss = 0.3400638699531555
ts 103 epoch = 99 loss = 0.3250678479671478
ts 104 epoch = 99 loss = 0.48220309615135193
ts 105 epoch = 99 loss = 0.31326165795326233
ts 106 epoch = 99 loss = 0.31326189637184143
ts 107 epoch = 99 loss = 0.31425711512565613
ts 108 epoch = 99 loss = 0.31495967507362366
ts 109 epoch = 99 loss = 0.3143446147441864
ts 110 epoch = 99 loss = 0.31372329592704773
ts 111 epoch = 99 loss = 0.31326165795326233
ts 112 epoch = 99 loss = 0.31326165795326233
ts 113 epoch = 99 loss = 0.31327030062675476
ts 114 epoch = 99 loss = 0.31802237033843994
ts 115 epoch = 99 loss = 0.5632616281509399
ts 116 epoch = 99 loss = 0.41838932037353516
ts 117 epoch = 99 loss = 0.31736549735069275
ts 118 epoch = 99 loss = 0.4746730625629425
ts 119 epoch = 99 loss = 0.5129104852676392
ts 120 epoch = 99 loss = 0.31326165795326233
ts 121 epoch = 99 loss = 0.4371427595615387
ts 122 epoch = 99

ts 288 epoch = 99 loss = 0.3289801776409149
ts 289 epoch = 99 loss = 0.31326165795326233
ts 290 epoch = 99 loss = 0.4229678511619568
ts 291 epoch = 99 loss = 0.3186582922935486
ts 292 epoch = 99 loss = 0.5762646794319153
ts 1 epoch = 100 loss = 0.39803755283355713
ts 2 epoch = 100 loss = 0.31868302822113037
ts 3 epoch = 100 loss = 0.5132675170898438
ts 4 epoch = 100 loss = 0.31816309690475464
ts 5 epoch = 100 loss = 0.47846490144729614
ts 6 epoch = 100 loss = 0.3144208490848541
ts 7 epoch = 100 loss = 0.3685254752635956
ts 8 epoch = 100 loss = 0.31554728746414185
ts 9 epoch = 100 loss = 0.3136877715587616
ts 10 epoch = 100 loss = 0.31336280703544617
ts 11 epoch = 100 loss = 0.533577561378479
ts 12 epoch = 100 loss = 0.31791606545448303
ts 13 epoch = 100 loss = 0.6194907426834106
ts 14 epoch = 100 loss = 0.3137073516845703
ts 15 epoch = 100 loss = 0.31326165795326233
ts 16 epoch = 100 loss = 0.3132627010345459
ts 17 epoch = 100 loss = 0.3137096166610718
ts 18 epoch = 100 loss = 0.416030

ts 187 epoch = 100 loss = 0.31326165795326233
ts 188 epoch = 100 loss = 0.31457826495170593
ts 189 epoch = 100 loss = 0.47788333892822266
ts 190 epoch = 100 loss = 0.31811758875846863
ts 191 epoch = 100 loss = 0.31326165795326233
ts 192 epoch = 100 loss = 0.31326165795326233
ts 193 epoch = 100 loss = 0.5223742127418518
ts 194 epoch = 100 loss = 0.31326165795326233
ts 195 epoch = 100 loss = 0.31326165795326233
ts 196 epoch = 100 loss = 0.4390557110309601
ts 197 epoch = 100 loss = 0.3976259231567383
ts 198 epoch = 100 loss = 0.31631040573120117
ts 199 epoch = 100 loss = 0.3142823874950409
ts 200 epoch = 100 loss = 0.31326165795326233
ts 201 epoch = 100 loss = 0.31473004817962646
ts 202 epoch = 100 loss = 0.3149194121360779
ts 203 epoch = 100 loss = 0.3993951082229614
ts 204 epoch = 100 loss = 0.3152830898761749
ts 205 epoch = 100 loss = 0.3839501738548279
ts 206 epoch = 100 loss = 0.32149240374565125
ts 207 epoch = 100 loss = 0.3132617473602295
ts 208 epoch = 100 loss = 0.315809637308120

In [20]:
label_pred_list = []
y_true_list = []

def test():
    model.eval()
    with torch.no_grad():
        global label_pred_list
        global y_true_list
        for data in test_data:
            data.to('cpu')
            logits, _ = model(
                data.x.T.type(torch.FloatTensor),
                data.edge_attr.T.type(torch.FloatTensor),
                data.edge_to_edge_adj_matr.type(torch.FloatTensor),
                data.edge_to_node_adj_matr.type(torch.FloatTensor),
                data.node_to_edge_adj_matr.type(torch.FloatTensor),
                data.node_to_node_adj_matr.type(torch.FloatTensor)
            )
            label_pred = logits.max(1)[1].tolist()
            label_pred_list += label_pred
            y_true_list += data.y.tolist()
    model.train()

In [21]:
test()

In [22]:
import numpy as np
np.sum(label_pred_list)

200

In [23]:
label_pred_list

[0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,


In [24]:
y_true_list

[0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,


In [25]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [26]:
mean_precision = []
mean_recall = []
mean_f1 = []

prec = precision_score(y_true_list, label_pred_list, average='binary', labels=[0])
rec = recall_score(y_true_list, label_pred_list, average='binary', labels=[0])
f1 = f1_score(y_true_list, label_pred_list, average='binary', labels=[0])

In [27]:
print(classification_report(y_true_list,label_pred_list))

              precision    recall  f1-score   support

           0       0.96      0.94      0.95       409
           1       0.88      0.92      0.90       192

    accuracy                           0.93       601
   macro avg       0.92      0.93      0.92       601
weighted avg       0.93      0.93      0.93       601



In [28]:
prec

0.88

In [29]:
rec

0.9166666666666666

In [30]:
f1

0.8979591836734694

In [31]:
prec = precision_score(y_true_list, label_pred_list, average='macro')
rec = recall_score(y_true_list, label_pred_list, average='macro')
f1 = f1_score(y_true_list, label_pred_list, average='macro')

In [32]:
prec

0.9200498753117208

In [33]:
rec

0.9289934800325998

In [34]:
f1

0.9242882338120433

# Adicionando XGBoost

In [35]:
# Pesos saída

embeddings_train = []
y_train = []

model.eval()
with torch.no_grad():

    for data in train_data:
        data.to('cpu')
        _, embedding_train = model(
            data.x.T.type(torch.FloatTensor),
            data.edge_attr.T.type(torch.FloatTensor),
            data.edge_to_edge_adj_matr.type(torch.FloatTensor),
            data.edge_to_node_adj_matr.type(torch.FloatTensor),
            data.node_to_edge_adj_matr.type(torch.FloatTensor),
            data.node_to_node_adj_matr.type(torch.FloatTensor)
        )
        embeddings_train += embedding_train.numpy().tolist()
        y_train += data.y.numpy().tolist()
model.train()

Nenn(
  (layer1): EdgeLevelAttentionLayer(
    (importance_normalization): Softmax(dim=1)
    (node_activation): LeakyReLU(negative_slope=0.01)
    (edge_activation): LeakyReLU(negative_slope=0.01)
  )
  (layer2): NodeLevelAttentionLayer(
    (importance_normalization): Softmax(dim=0)
    (node_activation): LeakyReLU(negative_slope=0.01)
    (edge_activation): LeakyReLU(negative_slope=0.01)
  )
  (layer3): EdgeLevelAttentionLayer(
    (importance_normalization): Softmax(dim=1)
    (node_activation): LeakyReLU(negative_slope=0.01)
    (edge_activation): LeakyReLU(negative_slope=0.01)
  )
  (layer4): Linear(in_features=14, out_features=2, bias=True)
  (layer5): Softmax(dim=1)
)

In [36]:
embeddings_train = np.array(embeddings_train)

In [37]:
y_train = np.array(y_train)

In [38]:
embeddings_train.shape

(2470, 14)

In [39]:
y_train

array([1, 1, 0, ..., 0, 0, 0])

In [40]:
import xgboost as xgb

In [41]:
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)

In [42]:
xgb_model.fit(embeddings_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=42,
              reg_alpha=0, reg_lambda=1, ...)

In [43]:
embeddings_test = []
y_true = []

model.eval()
with torch.no_grad():

    for data in test_data:
        data.to('cpu')
        _, embedding_test = model(
            data.x.T.type(torch.FloatTensor),
            data.edge_attr.T.type(torch.FloatTensor),
            data.edge_to_edge_adj_matr.type(torch.FloatTensor),
            data.edge_to_node_adj_matr.type(torch.FloatTensor),
            data.node_to_edge_adj_matr.type(torch.FloatTensor),
            data.node_to_node_adj_matr.type(torch.FloatTensor)
        )
        embeddings_test += embedding_test.numpy().tolist()
        y_true += data.y.numpy().tolist()
model.train()

Nenn(
  (layer1): EdgeLevelAttentionLayer(
    (importance_normalization): Softmax(dim=1)
    (node_activation): LeakyReLU(negative_slope=0.01)
    (edge_activation): LeakyReLU(negative_slope=0.01)
  )
  (layer2): NodeLevelAttentionLayer(
    (importance_normalization): Softmax(dim=0)
    (node_activation): LeakyReLU(negative_slope=0.01)
    (edge_activation): LeakyReLU(negative_slope=0.01)
  )
  (layer3): EdgeLevelAttentionLayer(
    (importance_normalization): Softmax(dim=1)
    (node_activation): LeakyReLU(negative_slope=0.01)
    (edge_activation): LeakyReLU(negative_slope=0.01)
  )
  (layer4): Linear(in_features=14, out_features=2, bias=True)
  (layer5): Softmax(dim=1)
)

In [44]:
embeddings_test = np.array(embeddings_test)

In [45]:
y_true = np.array(y_true)

In [46]:
embeddings_test.shape

(601, 14)

In [47]:
y_true.shape

(601,)

In [48]:
y_pred = xgb_model.predict(embeddings_test)

In [49]:
print(classification_report(y_true,y_pred))

              precision    recall  f1-score   support

           0       0.96      0.94      0.95       409
           1       0.88      0.92      0.90       192

    accuracy                           0.93       601
   macro avg       0.92      0.93      0.92       601
weighted avg       0.93      0.93      0.93       601

